# Initiation

In [1]:
import boto3
import json
import os
import sys
import time
sys.path.append("../")

from dotenv import load_dotenv
from utils.openaiClient import OpenAIClient
from textractClient import (
    start_job,
    is_job_complete, 
    get_job_results,
    save_results_to_file,
    load_textract_pages,
    extract_tables_from_blocks_unmerged,
    extract_forms_kv,
    write_tables_csv,
    write_forms_csv,
    build_llm_jsonl
)

# Load the .env file (assuming it’s in the same folder)
load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")

boto3.setup_default_session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=AWS_REGION
)

openai_client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"))

# Extract from document

In [22]:
CATALOG_NAME = 'ms-02-02'
CATALOG_FILE = f'{CATALOG_NAME}.pdf'

S3_BUCKET = 'catalogs-for-extraction'
S3_CATALOG_PATH = f's3://{s3_bucket}/{document}'

TEXTRACT_OUTPUT_JSON_PATH = f"textract_output_{CATALOG_NAME}.json"

# job_id = start_job(s3_bucket, document, features=['TABLES'], region=AWS_REGION)
job_id = "bce4b0bc30439528968cc52ca5ae225868056799096afbcc6944bdaef4fe81df"

In [ ]:
status = is_job_complete(job_id, region=AWS_REGION)
results_pages = get_job_results(job_id, region=AWS_REGION)
save_results_to_file(results_pages, TEXTRACT_OUTPUT_JSON_PATH)

Job status: SUCCEEDED
Retrieved 1000 blocks on this page.
Fetching next page of results...
Retrieved 733 blocks on this page.
Saved results to textract_output_ms-02-02.json


# Parse table

In [34]:
pages = load_textract_pages(TEXTRACT_OUTPUT_JSON_PATH)

# Flatten all Blocks across responses (Textract paginates API, not your PDF pages)
all_blocks = []
for part in pages:
    all_blocks.extend(part.get("Blocks", []))

tables = extract_tables_from_blocks_unmerged(all_blocks, replicate_data=True, header_scan_rows=2)

In [37]:
table = tables[3]
table.keys()

dict_keys(['page', 'table_index', 'headers', 'rows', 'spans'])

In [38]:
for header in table['headers']:
    print(header)


Main Line Tubing Size / in.
Branch Swagelok Tube / Fitting Size
Orifice / in. (mm)
Ordering / Number
A
Dimensions, in. / B
(mm) / c
